In [ ]:
import numpy as np                             # linear algebra
import pandas as pd                            # data processing, CSV file I/O (e.g. pd.read_csv)
import re                                      # for Data Cleaning 
import matplotlib.pyplot as plt                # for Visualization  
import re                                      # for data Cleaning 

from tqdm import tqdm                          # for Progressear 
import nltk                                    # for preprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

from sklearn import metrics                    # for Accuracy Measure 
from sklearn.metrics import accuracy_score

In [ ]:
MBTI_CSV_CLEAN = 'clean_mbti_label.csv'

In [ ]:
df = pd.read_csv(MBTI_CSV_CLEAN)

In [ ]:
df

,label,comments,extraversion,sensitivity,feel,judge
0,INTJ,nowing youre in INTJ is a tool for you to use...,0,0,0,1
1,INTJ,You are truly an enlightened mastermind,0,0,0,1
2,INTP,watch a bit of everything including hentai I...,0,0,0,0
3,INTJ,dont know if I would count this as a pet pee...,0,0,0,1
4,INTP,,0,0,0,0
...,...,...,...,...,...,...
323019,INTP,he internet makes everything easier chat up a...,0,0,0,0
323020,INFP,but at the same time immense freedom and exci...,0,0,1,0
323021,INTJ,The great thing about SATW is that she explain...,0,0,0,1
323022,INFP,ine Lets do a dar,0,0,1,0


In [ ]:
df.rename(columns={'label': 'type', 'comments': 'post'}, inplace=True)

In [ ]:
df

,type,post,extraversion,sensitivity,feel,judge
0,INTJ,nowing youre in INTJ is a tool for you to use...,0,0,0,1
1,INTJ,You are truly an enlightened mastermind,0,0,0,1
2,INTP,watch a bit of everything including hentai I...,0,0,0,0
3,INTJ,dont know if I would count this as a pet pee...,0,0,0,1
4,INTP,,0,0,0,0
...,...,...,...,...,...,...
323019,INTP,he internet makes everything easier chat up a...,0,0,0,0
323020,INFP,but at the same time immense freedom and exci...,0,0,1,0
323021,INTJ,The great thing about SATW is that she explain...,0,0,0,1
323022,INFP,ine Lets do a dar,0,0,1,0


In [ ]:
df.value_counts()

type  post                                                                                                                                                                                                                                                                                                                                        extraversion  sensitivity  feel  judge
INTP                                                                                                                                                                                                                                                                                                                                              0             0            0     0        437
INTJ                                                                                                                                                                                                                                           

In [ ]:
#df.label.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(df.post,df.type,test_size = 0.20)

In [ ]:
tmp = pd.DataFrame({'type':Y_train, 'post':X_train})

In [ ]:
tmp

,type,post
14537,INFJ,FYou are both young and I am inclined to bel...
178884,INTP,How does someone acquire this motivation
5723,INTP,Same for me
146571,INTJ,dont ever turn my brain off and yes I have n...
295893,ENTP,INTJ5 weeks prematureintj\nINTPI prefer any f...
...,...,...
93761,INFJ,Ill be there Then get angry when I dont show ...
302092,INTJ,uh me not by much though 2 weeks int
132472,INTJ,His main idea is that life is meaningless Do w...
291761,ISTP,Java forever


In [ ]:
tmp.type.value_counts()

INTP    72438
INTJ    58100
INFP    28700
INFJ    28668
ENTP    25775
ENFP    15817
ISTP     7979
ENTJ     7267
ENFJ     3015
ISTJ     2738
ESTP     2105
ISFP     2038
ISFJ     1218
ESFP     1176
ESTJ      840
ESFJ      545
Name: type, dtype: int64

In [ ]:
# This Function Will be used to make data equal for all Class 
def up_down_sampling(data,count):
    types = list(set(data.type))
    defined = False
    for tp in types :
        print(tp)
        if not defined:
            defined = True
            tp_class_count = data.type.value_counts()[tp]
            if tp_class_count > count :
                df = data[data['type'] == tp].sample(count)
            else:
                df = data[data['type'] == tp].sample(count,replace = True)
        else:
            tp_class_count = data.type.value_counts()[tp]
            if tp_class_count > count :
                df = pd.concat([df, data[data['type'] == tp].sample(count)], axis=0)
            else:
                df = pd.concat([df, data[data['type'] == tp].sample(count,replace = True)], axis=0)
    return df

In [ ]:
df = up_down_sampling(tmp,25000)

ENFJ
ESFP
ENTJ
INTP
INTJ
ISFP
ESTJ
ISFJ
ESFJ
ISTP
INFJ
ENTP
ISTJ
INFP
ESTP
ENFP


In [ ]:
count_df = df.type.value_counts()
count_df

ISTJ    25000
ISFP    25000
ENFJ    25000
ESTP    25000
INFP    25000
ENFP    25000
ISTP    25000
ENTP    25000
ISFJ    25000
ENTJ    25000
INFJ    25000
ESFJ    25000
INTP    25000
ESTJ    25000
INTJ    25000
ESFP    25000
Name: type, dtype: int64

In [ ]:
X_train_S = df['post']
Y_train_S = df['type']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
vectorizer = CountVectorizer(ngram_range = (1,1))

In [ ]:
vectorizer.fit(X_train_S)

CountVectorizer()

In [ ]:
X = vectorizer.transform(X_train_S)

In [ ]:
Y = np.array(Y_train_S)

In [ ]:
X_t = vectorizer.transform(X_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
classifier_1 = MultinomialNB()

In [ ]:
classifier_1.fit(X,Y)

MultinomialNB()

In [ ]:
result_1 = classifier_1.predict(X_t)

In [ ]:
print("Accuracy Of Model 1 is :",accuracy_score(result_1,Y_test)*100)

Accuracy Of Model 1 is : 26.888011763795372


In [ ]:
from sklearn import tree

In [ ]:
classifier_2 = tree.DecisionTreeClassifier()

In [ ]:
classifier_2.fit(X,Y)

DecisionTreeClassifier()

In [ ]:
result_2 = classifier_2.predict(X_t)

In [ ]:
print("Accuracy Of Model 2 is :",accuracy_score(result_2,Y_test)*100)

Accuracy Of Model 2 is : 23.108118566674406


In [ ]:
! pip install catboost

In [ ]:
cat_features = 1000

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=50,
    random_seed=42,
    learning_rate=0.5,
    #custom_loss=['AUC' , 'Accuracy']
)

clf.fit(
    X, Y,
    cat_features=cat_features,
    eval_set=(X_t, Y_test),
    verbose=False,
    plot=True
)

In [ ]:
import xgboost as xgb

In [ ]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X, Y)
result_5 = gbm.predict(X_t)

In [ ]:
print("Accuracy Of Model 2 is :",accuracy_score(result_5,Y_test)*100)

Accuracy Of Model 2 is : 24.080179552666202
